In [1]:
import hopsworks
import pandas as pd
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os
from sklearn.ensemble import RandomForestRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194714
Connected. Call `.close()` to terminate connection gracefully.


#### Get or Create a new feature view

In [3]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
flight_fg = fs.get_feature_group(name="flight_data", version=1)
query = flight_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="flight_data",
                                  version=1,
                                  description="Read from Flight Delay dataset",
                                  labels=["dep_delay_new"],
                                  query=query)

#### Create a new train test split

In [13]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

#### Read from existing train test split

In [3]:
#flight_fg = fs.get_feature_group(name="flight_data", version=1)
#query = flight_fg.select_all()
feature_view = fs.get_feature_view(name="flight_data",version=1)
# get a training dataset with train and test splits
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=2)

Finished: Reading data from Hopsworks, using ArrowFlight (18.85s) 


In [4]:
X_train

,year,quarter,month,day_of_month,day_of_week,origin_airport_id,origin_wac,dest_airport_id,dest_wac,crs_dep_time,...,distance,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlystationpressure,hourlyvisibility,hourlywinddirection,hourlywindgustspeed,hourlywindspeed
0,2021,1,1,17,7,12266,74,12264,38,1255,...,1190.0,-2.2,17.8,0.00,26.0,1015.92,16.00,260,0.0,4.0
1,2022,2,5,27,5,13930,41,12478,22,730,...,740.0,9.4,12.8,0.07,80.0,984.08,16.09,10,13.0,8.0
3,2022,3,9,16,5,14100,23,10397,34,610,...,666.0,10.0,15.6,0.00,70.0,1022.69,16.09,360,0.0,3.0
4,2021,1,1,20,3,11433,43,14747,93,1946,...,1927.0,-8.3,-3.9,0.00,72.0,991.87,12.01,210,0.0,8.0
5,2021,1,2,1,1,11057,36,11292,82,914,...,1337.0,1.1,2.2,0.25,93.0,981.04,8.05,350,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663623,2021,4,11,19,5,11697,33,14100,23,2120,...,992.0,22.8,23.9,0.25,94.0,1017.95,16.09,40,9.0,6.0
1663624,2021,2,4,27,2,10397,34,11618,21,1859,...,746.0,14.4,24.4,0.00,54.0,981.71,16.09,190,0.0,2.0
1663625,2022,2,5,1,7,11433,43,11618,21,1932,...,488.0,7.2,15.0,0.00,60.0,988.15,16.00,270,0.0,8.0
1663627,2021,2,5,4,2,11292,82,12889,85,2245,...,628.0,3.9,7.8,0.00,77.0,834.74,16.09,240,0.0,4.0


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330904 entries, 0 to 1663628
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   year                       1330904 non-null  int64  
 1   quarter                    1330904 non-null  int64  
 2   month                      1330904 non-null  int64  
 3   day_of_month               1330904 non-null  int64  
 4   day_of_week                1330904 non-null  int64  
 5   origin_airport_id          1330904 non-null  int64  
 6   origin_wac                 1330904 non-null  int64  
 7   dest_airport_id            1330904 non-null  int64  
 8   dest_wac                   1330904 non-null  int64  
 9   crs_dep_time               1330904 non-null  int64  
 10  crs_arr_time               1330904 non-null  int64  
 11  distance                   1330904 non-null  float64
 12  hourlydewpointtemperature  1330904 non-null  float64
 13  hourlydrybulbtemp

In [6]:
y_train

,dep_delay_new
0,0.0
1,0.0
3,0.0
4,13.0
5,0.0
...,...
1663623,29.0
1663624,0.0
1663625,0.0
1663627,17.0


In [7]:
y_train.values.ravel()

array([ 0.,  0.,  0., ...,  0., 17.,  7.])

#### Random Forest Regressor Model

In [9]:
# Initialize the Random Forest classifier
model_RF = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF.fit(X_train, y_train)
# Predict labels for the test set
y_pred = model_RF.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 47.61099858428782
MAE: 20.013136475873523
R-squared: 0.04329396070047287
Mean Squared Error (MSE): 2266.8071861930566


In [27]:
y_train_np=y_train.values.ravel()
X_train_np=X_train.values
X_test_np = X_test.values
y_test_np=y_test.values.ravel()

In [ ]:
# Initialize the Random Forest classifier
model_RF_np = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF_np.fit(X_train_np, y_train_np)
# Predict labels for the test set
y_pred_np = model_RF_np.predict(X_test_np)

mse = mean_squared_error(y_test_np, y_pred_np)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test_np, y_pred_np)

# Calculate R-squared
r2 = r2_score(y_test_np, y_pred_np)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

#### Linear Regression

In [8]:
# Initialize the Linear Regression model
model_lin_reg = LinearRegression()

# Fit the model on the training data
model_lin_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_lin_reg.predict(X_test)

# Calculate the Mean Squared Error (MSE) on test set

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 48.24382525231649
MAE: 20.763486617884652
R-squared: 0.01769262168364405
Mean Squared Error (MSE): 2327.4666749760504


#### Neural Network

In [10]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [11]:
# Normalize features (optional but often recommended for neural networks)
# outputs np arrays
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_test_scaled = scaler.transform(X_test_tensor)

In [12]:
# Convert the scaled NumPy arrays back to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [13]:
# Convert to PyTorch Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [15]:
# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_loader)}, Val Loss: {val_loss}")

Epoch 1/25, Train Loss: 2414.179655575127, Val Loss: 2288.087933034983


Epoch 2/25, Train Loss: 2395.509612217689, Val Loss: 2277.009979453493
Epoch 3/25, Train Loss: 2389.207758262638, Val Loss: 2274.483069688225
Epoch 4/25, Train Loss: 2385.995572095583, Val Loss: 2271.526142435868
Epoch 5/25, Train Loss: 2382.48301912281, Val Loss: 2270.995143140502
Epoch 6/25, Train Loss: 2379.0541609543766, Val Loss: 2268.240525297945
Epoch 7/25, Train Loss: 2377.393388300441, Val Loss: 2265.3625396351376
Epoch 8/25, Train Loss: 2376.361021510143, Val Loss: 2267.5097285338743
Epoch 9/25, Train Loss: 2374.955282018097, Val Loss: 2262.4256778723643
Epoch 10/25, Train Loss: 2373.9578139393693, Val Loss: 2261.2571596988328
Epoch 11/25, Train Loss: 2371.8483466185776, Val Loss: 2260.3783112577485
Epoch 12/25, Train Loss: 2370.7172489186896, Val Loss: 2258.124268567936
Epoch 13/25, Train Loss: 2369.383239423242, Val Loss: 2259.5705655503903
Epoch 14/25, Train Loss: 2368.5375877815673, Val Loss: 2257.1462949234424
Epoch 15/25, Train Loss: 2367.1014015346523, Val Loss: 2257.9

#### Save to Hopsworks Model Registry 

In [36]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="flight delay model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/flight_delay_model.pkl")
#fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
#input_schema = Schema(X_train_tensor)
#output_schema = Schema(y_train_tensor)
#model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
fligt_delay_model = mr.python.create_model(
    name="fligt_delay_model", 
    #metrics={"accuracy" : metrics['accuracy']},
    metrics={"val MSE" : 2252},
    #model_schema=model_schema,
    description="Flight Delay Predictor no schema"
)

# Upload the model to the model registry, including all files in 'model_dir'
fligt_delay_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 47815/47815 elapsed<00:15 remaining<00:001,  3.35it/s]
Model export complete: 100%|██████████| 6/6 [00:22<00:00,  3.76s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/194714/models/fligt_delay_model/2


Model(name: 'fligt_delay_model', version: 2)

In [24]:
X_train_tensor[0]
y_train_tensor[0]

tensor([0.])

In [25]:
y_train_tensor[0]-model(X_train_tensor[0])

tensor([-5.0753], grad_fn=<SubBackward0>)

In [30]:
X_train

,year,quarter,month,day_of_month,day_of_week,origin_airport_id,origin_wac,dest_airport_id,dest_wac,crs_dep_time,...,distance,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlystationpressure,hourlyvisibility,hourlywinddirection,hourlywindgustspeed,hourlywindspeed
0,2021,1,1,17,7,12266,74,12264,38,1255,...,1190.0,-2.2,17.8,0.00,26.0,1015.92,16.00,260,0.0,4.0
1,2022,2,5,27,5,13930,41,12478,22,730,...,740.0,9.4,12.8,0.07,80.0,984.08,16.09,10,13.0,8.0
3,2022,3,9,16,5,14100,23,10397,34,610,...,666.0,10.0,15.6,0.00,70.0,1022.69,16.09,360,0.0,3.0
4,2021,1,1,20,3,11433,43,14747,93,1946,...,1927.0,-8.3,-3.9,0.00,72.0,991.87,12.01,210,0.0,8.0
5,2021,1,2,1,1,11057,36,11292,82,914,...,1337.0,1.1,2.2,0.25,93.0,981.04,8.05,350,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663623,2021,4,11,19,5,11697,33,14100,23,2120,...,992.0,22.8,23.9,0.25,94.0,1017.95,16.09,40,9.0,6.0
1663624,2021,2,4,27,2,10397,34,11618,21,1859,...,746.0,14.4,24.4,0.00,54.0,981.71,16.09,190,0.0,2.0
1663625,2022,2,5,1,7,11433,43,11618,21,1932,...,488.0,7.2,15.0,0.00,60.0,988.15,16.00,270,0.0,8.0
1663627,2021,2,5,4,2,11292,82,12889,85,2245,...,628.0,3.9,7.8,0.00,77.0,834.74,16.09,240,0.0,4.0


In [31]:
model(X_train)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame